In [2]:
import gprof_log as gplog
import os
import re
import pandas as pd

def make_dir_output(output_path, satd, enc):
    if not os.path.isdir(os.path.join(output_path, satd)):
        os.mkdir(os.path.join(output_path, satd))
    if not os.path.isdir(os.path.join(output_path, satd, 'excel')):
        os.mkdir((os.path.join(output_path, satd, 'excel')))
    if not os.path.isdir(os.path.join(output_path, satd, 'csv')):
        os.mkdir((os.path.join(output_path, satd, 'csv')))
    if not os.path.isdir(os.path.join(output_path, satd, 'excel', enc)):
        os.mkdir(os.path.join(output_path, satd, 'excel', enc))
    if not os.path.isdir(os.path.join(output_path, satd, 'csv', enc)):
        os.mkdir(os.path.join(output_path, satd, 'csv', enc))
    
    return

ptrn = re.compile(r'log_(.+)_qp(\d{2})_(\w+)_.+RdCost(.+)_exec')

output_path = 'outcsv/'

if not os.path.isdir(output_path):
    os.mkdir(output_path)

directory = '../out/out/'

satd_list = [os.path.join(directory, f) for f in os.listdir(directory)]

for satd in satd_list:
    files_list = []
    for path, currentDirectory, files in os.walk(directory):
        for file in files:
            if file.endswith('.gplog') or file.endswith('txt'):
                files_list.append(os.path.join(path, file))

        data_list = []
        for file in files_list:
            name, qp, encoder, satd = ptrn.findall(file)[0]
            data_list.append(gplog.read_log(file, name, qp, encoder, satd))

        df = gplog.group_by_filename_2(data_list)

    for key in df.keys():
        data = df[key]    

        make_dir_output(output_path, satd, data['encoder'][0])

        data.to_csv(os.path.join(output_path, satd, 'csv', data['encoder'][0], data['fileName'][0]+'.csv'), index=False)
        data.to_excel(os.path.join(output_path, satd, 'excel', data['encoder'][0], data['fileName'][0]+'.xlsx'), index=False)
